<a href="https://colab.research.google.com/github/callor/Callor-Deep-Leaning-2022/blob/master/%EC%A3%BC%EA%B0%80%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import packages
import pandas as pd
import numpy as np

In [3]:
#to plot within notebook
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#setting figure size
from matplotlib.pylab import rcParams
# 차트의 기본 크기를 설정합니다. 
rcParams['figure.figsize'] = 8,4 #그림(figure)의 크기. (가로,세로) 인치 단위

#for normalizing data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

In [8]:
#read the file
# https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/03/NSE-TATAGLOBAL11.csv 에서 다운로드드
df = pd.read_csv('NSE-TATAGLOBAL11.csv')
#print the head
print(df.head())

         Date    Open    High     Low    Last   Close  Total Trade Quantity  \
0  2018-10-08  208.00  222.25  206.85  216.00  215.15             4642146.0   
1  2018-10-05  217.00  218.60  205.90  210.25  209.20             3519515.0   
2  2018-10-04  223.50  227.80  216.15  217.25  218.20             1728786.0   
3  2018-10-03  230.00  237.50  225.75  226.45  227.60             1708590.0   
4  2018-10-01  234.55  234.60  221.05  230.30  230.90             1534749.0   

   Turnover (Lacs)  
0         10062.83  
1          7407.06  
2          3815.79  
3          3960.27  
4          3486.05  


In [9]:
#importing required libraries
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

#creating dataframe
# index를 sort함 sort 의미는 없음 
data = df.sort_index(ascending=True, axis=0)
print(data.head())

         Date    Open    High     Low    Last   Close  Total Trade Quantity  \
0  2018-10-08  208.00  222.25  206.85  216.00  215.15             4642146.0   
1  2018-10-05  217.00  218.60  205.90  210.25  209.20             3519515.0   
2  2018-10-04  223.50  227.80  216.15  217.25  218.20             1728786.0   
3  2018-10-03  230.00  237.50  225.75  226.45  227.60             1708590.0   
4  2018-10-01  234.55  234.60  221.05  230.30  230.90             1534749.0   

   Turnover (Lacs)  
0         10062.83  
1          7407.06  
2          3815.79  
3          3960.27  
4          3486.05  


In [10]:
#setting index
#Date 칼럼을 index 로 설정
data.index = data.Date

#그러면 date는 필요없어지니 삭제합니다
data.drop('Date', axis=1, inplace=True)

# numpy로 변환합니다. 1차원인 Close만 남습니다
dataset = data.values

train = dataset[0:987,:]
valid = dataset[987:,:]
# 각각 아래 데이터가 생성 되었습니다.
#(987, 1)
#(248, 1)
print(train.shape)
print(valid.shape)


(987, 7)
(248, 7)


In [11]:
# converting dataset into x_train and y_train
#값을 0 ~ 1 사이의 값으로 변환 합니다
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])

# x_train [0:60]=>0:59까지데이터 y_train [60]
x_train, y_train = np.array(x_train), np.array(y_train)
#(927, 60) (927,)
# x_train 60깨 input으로 가지는 데이터가 927개 들어있다.
print(x_train.shape,y_train.shape)
print(x_train)    

(927, 60) (927,)
[[0.46728972 0.50734312 0.53627058 ... 0.74499332 0.76457499 0.7694704 ]
 [0.50734312 0.53627058 0.56519804 ... 0.76457499 0.7694704  0.73542501]
 [0.53627058 0.56519804 0.58544726 ... 0.7694704  0.73542501 0.7423231 ]
 ...
 [0.24276814 0.23742768 0.22919448 ... 0.229417   0.22140632 0.23609257]
 [0.23742768 0.22919448 0.20872274 ... 0.22140632 0.23609257 0.24655096]
 [0.22919448 0.20872274 0.19670672 ... 0.23609257 0.24655096 0.25189141]]


In [12]:
# 입력 format에 맞게 변환 합니다.
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
print("input_shape")
print(x_train.shape,y_train.shape)
print(x_train)

input_shape
(927, 60, 1) (927,)
[[[0.46728972]
  [0.50734312]
  [0.53627058]
  ...
  [0.74499332]
  [0.76457499]
  [0.7694704 ]]

 [[0.50734312]
  [0.53627058]
  [0.56519804]
  ...
  [0.76457499]
  [0.7694704 ]
  [0.73542501]]

 [[0.53627058]
  [0.56519804]
  [0.58544726]
  ...
  [0.7694704 ]
  [0.73542501]
  [0.7423231 ]]

 ...

 [[0.24276814]
  [0.23742768]
  [0.22919448]
  ...
  [0.229417  ]
  [0.22140632]
  [0.23609257]]

 [[0.23742768]
  [0.22919448]
  [0.20872274]
  ...
  [0.22140632]
  [0.23609257]
  [0.24655096]]

 [[0.22919448]
  [0.20872274]
  [0.19670672]
  ...
  [0.23609257]
  [0.24655096]
  [0.25189141]]]


In [13]:
# create and fit the LSTM network
model = Sequential()
# x_train.shape[1] 이것은 timestep이 된다
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)


927/927 - 12s - loss: 0.0033 - 12s/epoch - 13ms/step


In [ ]:
# 입력 데이터중 마지막 데이터를 가지고 valid 데이터를 만듭니다.
#predicting 246 values, using past 60 from the train data
inputs = data[len(data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

In [15]:
X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [ ]:
closing_price = model.predict(X_test)
print(closing_price.shape)
closing_price = scaler.inverse_transform(closing_price)

In [ ]:
rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))
print(rms)

In [ ]:
#for plotting
train = new_data[:987]
valid = new_data[987:]
valid['Predictions'] = closing_price
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.show()